***GENERATED CODE FOR testpipeline315 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'customerID', 'transformation_label': 'String Indexer'}], 'feature': 'customerID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '99', 'mean': '', 'stddev': '', 'min': '0030-FNXPP', 'max': '9959-WOFKT', 'missing': '0', 'distinct': '99'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'customerID'}, {'feature_label': 'customerID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('customerID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'gender', 'transformation_label': 'String Indexer'}], 'feature': 'gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'gender'}, {'feature_label': 'gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Partner', 'transformation_label': 'String Indexer'}], 'feature': 'Partner', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Partner'}, {'feature_label': 'Partner', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Partner')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Dependents', 'transformation_label': 'String Indexer'}], 'feature': 'Dependents', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Dependents'}, {'feature_label': 'Dependents', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Dependents')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PhoneService', 'transformation_label': 'String Indexer'}], 'feature': 'PhoneService', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PhoneService'}, {'feature_label': 'PhoneService', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PhoneService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MultipleLines', 'transformation_label': 'String Indexer'}], 'feature': 'MultipleLines', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MultipleLines'}, {'feature_label': 'MultipleLines', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MultipleLines')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternetService', 'transformation_label': 'String Indexer'}], 'feature': 'InternetService', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'DSL', 'max': 'No', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternetService'}, {'feature_label': 'InternetService', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternetService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OnlineSecurity', 'transformation_label': 'String Indexer'}], 'feature': 'OnlineSecurity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OnlineSecurity'}, {'feature_label': 'OnlineSecurity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OnlineSecurity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OnlineBackup', 'transformation_label': 'String Indexer'}], 'feature': 'OnlineBackup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OnlineBackup'}, {'feature_label': 'OnlineBackup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OnlineBackup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DeviceProtection', 'transformation_label': 'String Indexer'}], 'feature': 'DeviceProtection', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DeviceProtection'}, {'feature_label': 'DeviceProtection', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DeviceProtection')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TechSupport', 'transformation_label': 'String Indexer'}], 'feature': 'TechSupport', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TechSupport'}, {'feature_label': 'TechSupport', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TechSupport')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StreamingTV', 'transformation_label': 'String Indexer'}], 'feature': 'StreamingTV', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StreamingTV'}, {'feature_label': 'StreamingTV', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StreamingTV')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StreamingMovies', 'transformation_label': 'String Indexer'}], 'feature': 'StreamingMovies', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StreamingMovies'}, {'feature_label': 'StreamingMovies', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StreamingMovies')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Contract', 'transformation_label': 'String Indexer'}], 'feature': 'Contract', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'Month-to-month', 'max': 'Two year', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Contract'}, {'feature_label': 'Contract', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Contract')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PaperlessBilling', 'transformation_label': 'String Indexer'}], 'feature': 'PaperlessBilling', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PaperlessBilling'}, {'feature_label': 'PaperlessBilling', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PaperlessBilling')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PaymentMethod', 'transformation_label': 'String Indexer'}], 'feature': 'PaymentMethod', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'Bank transfer (automatic)', 'max': 'Mailed check', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PaymentMethod'}, {'feature_label': 'PaymentMethod', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PaymentMethod')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Churn', 'transformation_label': 'String Indexer'}], 'feature': 'Churn', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '99', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Churn'}, {'feature_label': 'Churn', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Churn')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run testpipeline315Hooks.ipynb
try:
	#sourcePreExecutionHook()

	testpipelinehdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1731435912.csv', 'filename': 'tempFile1731435912.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(testpipelinehdfs)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run testpipeline315Hooks.ipynb
try:
	#transformationPreExecutionHook()

	testpipelineautofe = TransformationMain.run(testpipelinehdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "customerID", "transformation_label": "String Indexer"}], "feature": "customerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "0030-FNXPP", "max": "9959-WOFKT", "missing": "0", "distinct": "99"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "customerID"}, {"transformationsData": [{"feature_label": "gender", "transformation_label": "String Indexer"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SeniorCitizen", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "0.13", "stddev": "0.34", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "SeniorCitizen"}, {"transformationsData": [{"feature_label": "Partner", "transformation_label": "String Indexer"}], "feature": "Partner", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Partner"}, {"transformationsData": [{"feature_label": "Dependents", "transformation_label": "String Indexer"}], "feature": "Dependents", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Dependents"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tenure", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "29.44", "stddev": "25.1", "min": "1", "max": "72", "missing": "0"}, "updatedLabel": "tenure"}, {"transformationsData": [{"feature_label": "PhoneService", "transformation_label": "String Indexer"}], "feature": "PhoneService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PhoneService"}, {"transformationsData": [{"feature_label": "MultipleLines", "transformation_label": "String Indexer"}], "feature": "MultipleLines", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MultipleLines"}, {"transformationsData": [{"feature_label": "InternetService", "transformation_label": "String Indexer"}], "feature": "InternetService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "DSL", "max": "No", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternetService"}, {"transformationsData": [{"feature_label": "OnlineSecurity", "transformation_label": "String Indexer"}], "feature": "OnlineSecurity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OnlineSecurity"}, {"transformationsData": [{"feature_label": "OnlineBackup", "transformation_label": "String Indexer"}], "feature": "OnlineBackup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OnlineBackup"}, {"transformationsData": [{"feature_label": "DeviceProtection", "transformation_label": "String Indexer"}], "feature": "DeviceProtection", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DeviceProtection"}, {"transformationsData": [{"feature_label": "TechSupport", "transformation_label": "String Indexer"}], "feature": "TechSupport", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TechSupport"}, {"transformationsData": [{"feature_label": "StreamingTV", "transformation_label": "String Indexer"}], "feature": "StreamingTV", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StreamingTV"}, {"transformationsData": [{"feature_label": "StreamingMovies", "transformation_label": "String Indexer"}], "feature": "StreamingMovies", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StreamingMovies"}, {"transformationsData": [{"feature_label": "Contract", "transformation_label": "String Indexer"}], "feature": "Contract", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "Month-to-month", "max": "Two year", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Contract"}, {"transformationsData": [{"feature_label": "PaperlessBilling", "transformation_label": "String Indexer"}], "feature": "PaperlessBilling", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PaperlessBilling"}, {"transformationsData": [{"feature_label": "PaymentMethod", "transformation_label": "String Indexer"}], "feature": "PaymentMethod", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "Bank transfer (automatic)", "max": "Mailed check", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PaymentMethod"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthlyCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "65.91", "stddev": "30.23", "min": "19.15", "max": "115.05", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MonthlyCharges"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "2281.86", "stddev": "2359.46", "min": "19.15", "max": "7966.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TotalCharges"}, {"transformationsData": [{"feature_label": "Churn", "transformation_label": "String Indexer"}], "feature": "Churn", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Churn"}]}))

	#transformationPostExecutionHook(testpipelineautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run testpipeline315Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(testpipelineautofe, ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges", "customerID_stringindexer", "gender_stringindexer", "Partner_stringindexer", "Dependents_stringindexer", "PhoneService_stringindexer", "MultipleLines_stringindexer", "InternetService_stringindexer", "OnlineSecurity_stringindexer", "OnlineBackup_stringindexer", "DeviceProtection_stringindexer", "TechSupport_stringindexer", "StreamingTV_stringindexer", "StreamingMovies_stringindexer", "Contract_stringindexer", "PaperlessBilling_stringindexer", "PaymentMethod_stringindexer"], "Churn_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

